# Linguistische Annotation von Texten

Die linguistische Annotation von Texten meint insbesondere die folgenden Schritte, die oft ein Teil des Präprozessierens von Texten für die weiterführenden Analysen sind: 

1. Tokenisierung
2. Lemmatisierung
3. POS-Tagging
4. Named Entity Recognition

Es gibt für die linguistische Annotation eine ganze Reihe von einschlägigen Tools, die Sie kennen sollten: 

1. NLTK (Natural Language Tool-Kit): 
1. Stanford NLP
1. spaCy 

Heute fokussieren wir auf spaCy, weil das ein relativ neues, modernes, sehr aktiv entwickeltes und für diverse Sprachen geeignetes Annotationstool ist. Beispielsweise sind auch Methoden verfügbar, die auf neuronalen Netzen basieren (Deep Learning) oder können ggfs. vorhandene GPUs beim Prozessieren genutzt werden. 

* Homepage: https://spacy.io/
* Dokumentation zur Annotation: https://spacy.io/usage/linguistic-features

Als Bonus können wir mit spaCy auch recht einfach Wortvektoren abrufen, die die Semantik der Wörter im Text repräsentieren. Damit kann man beispielsweise sehr einfach Wörter mit ähnlicher Bedeutung (Synonyme) in einem Text identifizieren. 

## Korpus

Wir verwenden als Beispielkorpus die drei kurzen deutschen Erzähltexte, die unter dem Titel "narration" im `datasets`-Repository verfügbar sind: https://github.com/dh-trier/datasets/ (im Ordner "corpora").  

## Installation von spaCy

Informationen zur Installation: https://spacy.io/usage

Deutsche Annotationsmodelle:

* auf der Kommandozeile (effizientes Modell): `python3 -m spacy download de_core_news_sm` 
* Auf accuracy optimiertes Modell: `python3 -m spacy download de_dep_news_trf`. 
* Ein besonders umfangreiches Modell mit sehr vielen Wortvektoren: `python3 -m spacy download de_core_news_lg`. 

## Importe

In [5]:
# Basisimporte
from os.path import join
from os.path import basename
from glob import glob

# Spezielle Importe
import spacy
import wikipedia
import pandas as pd
from itertools import combinations

## Texte laden

In [6]:
corpusfolder = join("..", "..", "datasets", "corpora", "narration", "*.txt")

def load_text(textfile): 
    with open(textfile, "r", encoding="utf8") as infile: 
        text = infile.read()
    return text

def main(corpusfolder): 
    corpus = {}
    for textfile in glob(corpusfolder): 
        idno = basename(textfile).split(".")[0]
        corpus[idno] = load_text(textfile)
    for idno,text in corpus.items(): 
        print(idno, text[0:50])
    return corpus

corpus = main(corpusfolder)

Tucholsky Bademeister! Bademeister! Unerhört! Ist die Ostsee
Münchhausen ## Wunderbare Reisen zu Wasser und zu Lande, Feldz
Kraus EIN MORD ist geschehen und die Menschheit möchte u


## spaCy verwenden

In [7]:
# Annotationspipeline mit spezifischem Modell initialisieren
nlp = spacy.load("de_core_news_sm")  # Oder: "de_dep_news_trf"

def annotate(nlp, text): 
    annotated = nlp(text)
    return annotated

def main(corpus, nlp): 
    acorpus = {}
    for idno,text in corpus.items(): 
        acorpus[idno] = nlp(text)
    return acorpus

acorpus = main(corpus, nlp)


## Auf Annotationen zugreifen

Ein annotierter Text ist in spaCy eine spezifische Struktur, nämlich eine Liste von Tokens, wobei jedes Token ein Objekt mit einer Reihe von Methoden ist. Diese Methoden erlauben den Zugriff auf die verschiedenen Annotationsebenen. Dazu gehören die folgenden Methoden bzw. Annotationen: 

* `token.text`: die Wortform
* `token.lemma_`: das Lemma
* `token.pos_`: das POS-Tag (einfaches UPOS Tag)
* `token.tag_`: das POS-Tag (vollständiges / detailliertes Tag)
* `token.dep_`: die syntaktische Annotation (dependency)
* `token.shape_`: Gibt es Kapitalisierung, Interpunktion, Zahlen?
* `token.is_alpha`: besteht das Token aus Buchstaben? (True/False)
* `token.is_stop`: ist das Token ein Stopword? (True/False)
* `token.morph`: detaillierte morphologische Information (u.a. Kasus, Genus, Numerus)

In [9]:
def show_annotations(atext): 
        for token in atext[10:15]: 
            print(token.text, "(lemma, pos, tag)", token.lemma_, token.pos_, token.tag_)
            print(token.text, "(shape, alpha, stop)", token.shape_, token.is_alpha, token.is_stop)
            print(token.text, "(morph)", token.morph)
            print(token.text, "(dep)", token.dep_, "\n")


def get_annotation(atext):
    alist = [token.pos_ for token in atext]
    print(alist[0:40])
    return alist
                      
            
def retrieve_annotations(acorpus): 
    for idno,atext in acorpus.items(): 
        print("===\nText:", idno)
        #show_annotations(atext)         # Für die visuelle Inspektion
        alist = get_annotation(atext)   # Um eine Annotationsschicht weiter zu verarbeiten

retrieve_annotations(acorpus)

===
Text: Tucholsky
['NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'ADV', 'PUNCT', 'AUX', 'DET', 'PROPN', 'ADP', 'DET', 'NOUN', 'ADV', 'PUNCT', 'CCONJ', 'AUX', 'PRON', 'ADP', 'DET', 'PROPN', 'ADV', 'PUNCT', 'PRON', 'PRON', 'DET', 'NOUN', 'VERB', 'PUNCT', 'AUX', 'VERB', 'PRON', 'ADV', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADV', 'PUNCT', 'PART', 'PUNCT']
===
Text: Münchhausen
['PROPN', 'PROPN', 'PROPN', 'NOUN', 'ADP', 'NOUN', 'CCONJ', 'ADP', 'NOUN', 'PUNCT', 'NOUN', 'CCONJ', 'ADJ', 'NOUN', 'DET', 'NOUN', 'ADP', 'NOUN', 'SPACE', 'PROPN', 'NOUN', 'SPACE', 'NOUN', 'NOUN', 'PROPN', 'PROPN', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'NOUN', 'NOUN', 'NOUN', 'SPACE', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'NOUN']
===
Text: Kraus
['DET', 'NOUN', 'AUX', 'VERB', 'CCONJ', 'DET', 'NOUN', 'AUX', 'ADP', 'NOUN', 'VERB', 'PUNCT', 'PRON', 'AUX', 'PRON', 'PART', 'PUNCT', 'PRON', 'PUNCT', 'DET', 'NOUN', 'PUNCT', 'ADV', 'ADV', 'PUNCT', 'ADP', 'PRON', 'ADJ', 'NOUN', 'DET', 'ADJ', 'NOUN', 'PART', 'VERB', 'PUNCT', 'PRON', 'PUNCT', 'PRON', '

## Named Entities

Auch die Named Entities sind in der Standard-Annotation direkt enthalten, allerdings in einem Unter-Objekt, `.ent`. Hier sind nur diejenigen Token enthalten, die auch tatsächlich als Entität erkannt wurden. 

Die folgenden Eigenschaften können abgerufen werden: 

* `ent.text`: Wortform
* `ent.label_`: NE-Kategorie
* `ent.start_char`: Position des Anfangszeichens im Text
* `ent.end_char`: Position des letzten Zeichens im Text


In [10]:
def show_entities(atext): 
        for ent in atext.ents[10:15]: 
            print(ent.text, ent.label_, ent.start_char, ent.end_char)
                   
            
def retrieve_entities(acorpus): 
    for idno,atext in acorpus.items(): 
        print("===\nText:", idno)
        show_entities(atext) 

retrieve_entities(acorpus)

===
Text: Tucholsky
Hab PER 847 850
Nein LOC 869 873
Gumpel PER 958 964
Ufa ORG 1005 1008
Höchster PER 1019 1027
===
Text: Münchhausen
Betrügereyen LOC 2241 2253
deutschen MISC 2609 2618
deutschen MISC 2662 2671
Tractatus PER 3169 3178
Commentarius PER 3185 3197
===
Text: Kraus
Opiumnebel LOC 1499 1509
Kinn LOC 1718 1722


Tja, man sieht dass das nicht besonders gut funktioniert. Es dürfte daran liegen, dass das Modell mit modernen Nachrichtentexten trainiert wurde, während wir hier eben fiktionale Erzähltexte aus dem frühen 20. Jahrhundert vorliegen haben. 

## Kleiner Test auf einem Wikipedia-Artikel

1. Wir laden mit dem Modul `wikipedia` einen Artikel aus Wikipedia herunter, direkt als plain text. Siehe die Dokumentation hier: https://wikipedia.readthedocs.io/en/latest/quickstart.html#quickstart
2. Wir annotieren den Text und lassen uns dann die Named Entities herausgeben. 

(Eine Alternative für den Download des Textes wäre natürlich auch `requests`, mit dem man die API von Wikipedia auch gut nutzen kann. Siehe die Hinweise hier: https://stackoverflow.com/questions/4452102/how-to-get-plain-text-out-of-wikipedia). 


In [18]:
wptitle = "Riesling"    # "Riesling", Paul Otlet", "Alan Turing" (?), "Susan Sontag", "Mosel" (?)
lang = "de"

def get_wptext(wptitle, lang): 
    """
    Lade einen bestimmten Wikipedia-Artikel in einer bestimmten Sprache herunter. 
    Gibt den plain text des vollständigen Artikels zurück. 
    """
    wikipedia.set_lang(lang)
    resp = wikipedia.page(wptitle)
    print(resp.title, resp.url)
    return resp.content

def annotate_wptext(wptext): 
    """
    Annotiere den Wikipedia-Artikel. 
    Zeige die Named Entities. 
    """
    atext = nlp(wptext)
    for ent in atext.ents: 
        print(ent.text, ent.label_,)

        
def main(wptitle, lang): 
    wptext = get_wptext(wptitle, lang)
    annotate_wptext(wptext)
    #print(wptext)

main(wptitle, lang)

Riesling https://de.wikipedia.org/wiki/Riesling
Deutschland LOC
Riesling-Weine MISC
Deutschland LOC
Deutschland LOC
Deutschland LOC
Heunisch LOC
Wildrebenbeständen LOC
Oberrhein LOC
Ferdinand Regner PER
Höheren Bundeslehranstalt für Wein- und Obstbau ORG
Klosterneuburg LOC
Traminer LOC
Rhein LOC
Traminer LOC
Römer LOC
Rhein LOC
Wildreben LOC
Rhein LOC
Rheintal LOC
deutschen MISC
Riesling-Reben LOC
Jahr 1402 MISC
Worms LOC
Rüsselsheimer LOC
Klaus Kleinfisch PER
Johann IV. von Katzenelnbogen PER
Marburg LOC
Rüsselsheim am Main LOC
Rüsselsheimer LOC
Jahr 1401 MISC
Zwingenberg LOC
Auerbach LOC
Rüsselsheim LOC
Bingen LOC
Mosel LOC
lateinischen MISC
Hieronymus Bock PER
Rüsselsheim LOC
Ersten Weltkrieges MISC
Rüsselsheim LOC
Wingert LOC
Rüsselsheim LOC
Gemeinde LOC
Museumskeller LOC
Festung Rüsselsheim LOC
Rüsselsheimer Riesling ORG
Rheingauer Weinberge LOC
Schloss Johannisberg LOC
Fuldaer LOC
Trierer LOC
Clemens Wenzeslaus von Sachsen PER
Mosel LOC
Wende MISC
deutsche MISC
Frankreichs LOC
Tr

In der Tat sieht man hier sehr schön, dass das mit einem solchen Sachtext sehr gut funktioniert. 

## Wortvektoren: Semantische Ähnlichkeit

Wir laden noch einmal einen Wikipedia-Artikel aus dem obigen Beispiel und suchen jetzt in diesem Text Paare von Wörtern, die sich besonders ähnlich sind.

Damit das gut funktioniert, brauchen wir ein größeres Sprachmodell, das auch (viele) Wortvektoren enthält. Daher bitte noch das "de_core_news_lg" herunterladen, wie oben.

In [30]:
nlp = spacy.load("de_core_news_lg")
wikipedia.set_lang("de")
text = wikipedia.page("Riesling").content
atext = nlp(text)

In [33]:
def filter_atext(atext): 
    wordlist = []
    vectors = []
    for token in atext: 
        if token.is_oov == False: 
            if token.pos_ == "NOUN": # VERB|NOUN
                if token.text not in wordlist:     # Wir wollen Duplikate der Wortformen vermeiden
                    wordlist.append(token.text)
                    vectors.append(token)
    return vectors

def get_similarities(vectors):
    pairs = set(combinations(vectors, 2))
    results = {}
    for item in pairs: 
        results[item[0].text + " ~ " + item[1].text] = item[0].similarity(item[1])
    results = pd.Series(results)
    results.sort_values(ascending=False, inplace=True)
    return results

def find_similar_words(atext): 
    vectors = filter_atext(atext)
    results = get_similarities(vectors)
    
    print("Most similar word pairs:")
    print(results.head(30))
    print("\nLeast similar word pairs:")
    print(results.tail(10))

find_similar_words(atext)

Most similar word pairs:
Rotweine ~ Weißweine                0.968278
Neuzüchtungen ~ Züchtungen          0.950376
Jahrhundert ~ Jahrhunderts          0.945154
Weißweinsorte ~ Weißweinsorten      0.939560
Weinbaugebieten ~ Anbaugebieten     0.935008
Weinanbau ~ Weinbau                 0.928175
Anbaugebiete ~ Anbaugebieten        0.922332
Weißweinsorten ~ Rebsorten          0.911098
Weinsorte ~ Weißweinsorte           0.909457
Qualitätsweinen ~ Qualitätsweine    0.905487
Weingärten ~ Weinberge              0.898923
Beeren ~ Früchten                   0.897639
Rebsorte ~ Weißweinsorte            0.897239
Rotweine ~ Weine                    0.895659
Rebsorte ~ Rebsorten                0.889420
Weinbaugebieten ~ Anbaugebiete      0.886419
Weine ~ Weißweine                   0.880512
Weinsorte ~ Weißweinsorten          0.873693
Jahren ~ Jahre                      0.863617
Anbaufläche ~ Rebfläche             0.863364
Weinsorte ~ Rebsorte                0.863353
ha ~ Hektar                   